In [1]:
import pandas as pd
from scipy import stats

# 파일 불러오기 및 기본 구조 확인
file_path = 'kosdaq_2010_1parm.csv' 
df = pd.read_csv(file_path, encoding='cp949')
print(df.head()) # 데이터의 처음 몇 행을 출력

# 기본 정보 및 결측치 확인
print(df.info()) # 데이터의 기본 정보 출력
print(df.isnull().sum()) # 각 컬럼별 결측치 수 확인

# 회사별 재무 상태 변화 분석
# 데이터를 회사명과 회계년도로 그룹화하여 평균값 계산
df_grouped = df.groupby(['회사명', '회계년도']).mean()
print(df_grouped.head()) # 결과 확인

# t-test 수행
# 2010년과 2019년 데이터를 각각 분리
df_2010 = df[df['회계년도'] == '2010/12']
df_2019 = df[df['회계년도'] == '2019/12']

# '매출액증가율(IFRS)'에 대한 t-test 수행
t_test_result = stats.ttest_ind(df_2010['매출액증가율(IFRS)'].dropna(), 
                                df_2019['매출액증가율(IFRS)'].dropna(), 
                                equal_var=False)
print(t_test_result) # t-test 결과 출력

# 파생변수 생성
# 데이터를 회사명으로 정렬
df_sorted = df.sort_values(['회사명', '회계년도'])

# '매출액순이익률(IFRS)'의 연도별 차이와 비율을 계산
df_sorted['매출액순이익률_diff'] = df_sorted.groupby('회사명')['매출액순이익률(IFRS)'].diff()
df_sorted['매출액순이익률_ratio'] = df_sorted.groupby('회사명')['매출액순이익률(IFRS)'].pct_change()
print(df_sorted[['회사명', '회계년도', '매출액순이익률_diff', '매출액순이익률_ratio']].head()) # 결과 확인

# 상위 10개 기업 선정 및 Gap Model 변수 생성
# '매출액순이익률(IFRS)' 기준 상위 10개 기업 추출
top_10_companies = df_sorted.groupby('회사명')['매출액순이익률(IFRS)'].mean().sort_values(ascending=False).head(10)

# 상위 10개 기업의 평균 계산
top_10_average = top_10_companies.mean()

# 산업 전체 평균 계산
industry_average = df_sorted['매출액순이익률(IFRS)'].mean()

# Gap Model 변수 생성
df_sorted['Top_10_Gap_Diff'] = df_sorted['매출액순이익률(IFRS)'] - top_10_average
df_sorted['Top_10_Gap_Ratio'] = df_sorted['매출액순이익률(IFRS)'] / top_10_average
df_sorted['Average_Gap_Diff'] = df_sorted['매출액순이익률(IFRS)'] - industry_average
df_sorted['Average_Gap_Ratio'] = df_sorted['매출액순이익률(IFRS)'] / industry_average
print(df_sorted[['회사명', '회계년도', '매출액순이익률(IFRS)', 'Top_10_Gap_Diff', 'Top_10_Gap_Ratio', 'Average_Gap_Diff', 'Average_Gap_Ratio']].head()) # 결과 확인


        회사명  거래소코드     회계년도  매출액증가율(IFRS)  매출액순이익률(IFRS)  총자본투자효율(IFRS)  \
0  (주)CMG제약  58820  2010/12         11.86         -61.19           2.98   
1  (주)CMG제약  58820  2011/12         56.40          -7.12          13.32   
2  (주)CMG제약  58820  2012/12         -8.43         -42.40          -0.15   
3  (주)CMG제약  58820  2013/12         25.02           1.65          12.42   
4  (주)CMG제약  58820  2014/12         11.96           0.75          12.15   

   CASH FLOW 대 부채비율(IFRS)  총자본회전률(IFRS)  
0                  -69.69          0.46  
1                  -45.76          0.65  
2                   -7.76          0.38  
3                  -38.65          0.36  
4                  -17.44          0.40  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15657 entries, 0 to 15656
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   회사명                     15657 non-null  object 
 1   거래소코드                   1

C:\Users\USER\AppData\Local\Temp\ipykernel_5876\84863193.py:35: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  df_sorted['매출액순이익률_ratio'] = df_sorted.groupby('회사명')['매출액순이익률(IFRS)'].pct_change()


In [2]:
# 파일을 다시 불러오기
df1_updated = pd.read_csv('kosdaq_2012_1parm.csv', encoding='cp949')
df2_updated = pd.read_csv('kosdaq_2012_1parm_live.csv', encoding='cp949')

# 각 파일에서 회사명만 추출
companies_1_updated = set(df1_updated['회사명'])
companies_2_updated = set(df2_updated['회사명'])

# 첫 번째 파일에만 있는 회사명 추출
unique_companies_in_1_not_in_2_updated = companies_1_updated - companies_2_updated

# 결과 출력
unique_companies_in_1_not_in_2_updated

set()

In [6]:
# 각 CSV 파일에서 '회사명'이 '(주)맘스터치앤컴퍼니'인 행 찾기
moms_touch_and_company_df1 = df1_updated[df1_updated['회사명'] == '(주)맘스터치앤컴퍼니']
moms_touch_and_company_df2 = df2_updated[df2_updated['회사명'] == '(주)맘스터치앤컴퍼니']

# 결과 확인
moms_touch_and_company_df1, moms_touch_and_company_df2

(Empty DataFrame
 Columns: [회사명, 거래소코드, 회계년도, 매출액증가율(IFRS), 매출액순이익률(IFRS), 총자본투자효율(IFRS), CASH FLOW 대 부채비율(IFRS), 총자본회전률(IFRS)]
 Index: [],
 Empty DataFrame
 Columns: [회사명, 거래소코드, 회계년도, 매출액증가율(IFRS), 매출액순이익률(IFRS), 총자본투자효율(IFRS), CASH FLOW 대 부채비율(IFRS), 총자본회전률(IFRS)]
 Index: [])

In [3]:
df1_updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   회사명                     804 non-null    object 
 1   거래소코드                   804 non-null    int64  
 2   회계년도                    804 non-null    object 
 3   매출액증가율(IFRS)            797 non-null    float64
 4   매출액순이익률(IFRS)           797 non-null    float64
 5   총자본투자효율(IFRS)           797 non-null    float64
 6   CASH FLOW 대 부채비율(IFRS)  797 non-null    float64
 7   총자본회전률(IFRS)            797 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 50.4+ KB


In [4]:
df2_updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   회사명                     804 non-null    object 
 1   거래소코드                   804 non-null    int64  
 2   회계년도                    804 non-null    object 
 3   매출액증가율(IFRS)            797 non-null    float64
 4   매출액순이익률(IFRS)           797 non-null    float64
 5   총자본투자효율(IFRS)           797 non-null    float64
 6   CASH FLOW 대 부채비율(IFRS)  797 non-null    float64
 7   총자본회전률(IFRS)            797 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 50.4+ KB
